# Part 1 - Fundamentals
## Chapter 3 - Lambda expressions
Gives a full explanation, with code examples and quizzes at every step, of the concepts of lambda expressions and method references.

In [1]:
System.out.println("Lets start with chapter 3, without wasting any time!")

Lets start with chapter 3, without wasting any time!


This chapter covers
* Lambdas in a nutshell
* Where and how to use lambdas
* Functional interfaces, type inference 
* Method references
* Composing lambdas

In [2]:
public static class Apple {

	private Integer weight = 0;
	private String color = "";

	public Apple(int weight, String color) {
		this.weight = weight;
		this.color = color;
	}
    
    public Apple(int weight){
        this.weight = weight;
    }
    
    public Apple(){
        this.weight = 0;
        this.color = "white";
    }

	public Integer getWeight() {
		return weight;
	}

	public void setWeight(int weight) {
		this.weight = weight;
	}

	public String getColor() {
		return color;
	}

	public void setColor(String color) {
		this.color = color;
	}

	@Override
	public String toString() {
		return String.format("Apple{color='%s', weight=%d}", color, weight);
	}
}

In [3]:
private List<Apple> getInventory() {
	var inventory = Arrays.asList(new Apple(80, "green"), 
                                  new Apple(255, "green"), 
                                  new Apple(220, "red"));
	return inventory;
}

## 3.1 Lambdas in a nutshell
A lambda expression can be understood as a concise representation of an anonymous function that can be passed around. It doesn’t have a name, but it has a list of parameters, a body, a return type, and also possibly a list of exceptions that can be thrown.

That’s one big definition; let’s break it down:
* Anonymous—We say anonymous because it doesn’t have an explicit name like a method would normally have; less to write and think about!
* Function—We say function because a lambda isn’t associated with a particular class like a method is. But like a method, a lambda has a list of parameters, a body, a return type, and a possible list of exceptions that can be thrown.
* Passed around—A lambda expression can be passed as argument to a method or stored in a variable.
* Concise—You don’t need to write a lot of boilerplate like you do for anonymous classes.

If you’re wondering where the term lambda comes from, it originates from a system developed in academia called lambda calculus, which is used to describe computations.

| Use case | Examples of lambdas |
| :--- | :--- |
| A boolean expression | (List\<String> list) -> list.isEmpty()
| Creating objects | () -> new Apple(10)
| Consuming from an object | (Apple a) -> { System.out.println(a.getWeight());}
| Select/extract from an object | (String s) -> s.length()
| Combine two values | (int a, int b) -> a * b
| Compare two objects | (Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight())

## 3.2 Where and how to use lambdas
Where exactly can you use lambdas? You can use a lambda expression in the context of a functional interface.

In a nutshell, a functional interface is an interface that specifies exactly one abstract method. You already know several other functional interfaces in the Java API such as Comparator and Runnable

_An interface is still a functional interface if it has many default methods as long as it specifies only one abstract method._

What can you do with functional interfaces? Lambda expressions let you provide the implementation of the abstract method of a functional interface directly inline and treat the whole expression as an instance of a functional interface (more technically speaking, an instance of a concrete implementation of the functional interface).

If you explore the new Java API, you will notice that functional interfaces are generally annotated with @FunctionalInterface.

## 3.3 Using functional interfaces

### Predicate

The java.util.function.Predicate\<T> interface defines an abstract method named test that accepts an object of generic type T and returns a boolean.

You might want to use this interface when you need to represent a boolean expression that uses an object of type T.

In [4]:
@FunctionalInterface
public interface Predicate<T> {
	boolean test(T t);
}

public <T> List<T> filter(List<T> list, Predicate<T> p) {
	List<T> results = new ArrayList<>();
	for (T t : list) {
		if (p.test(t)) {
			results.add(t);
		}
	}
	return results;
}

In [5]:
Predicate<String> nonEmptyStringPredicate = (String s) -> !s.isEmpty();

In [6]:
List listOfStrings = List.of("","test","another test");
List<String> nonEmpty = filter(listOfStrings, nonEmptyStringPredicate);

nonEmpty

[test, another test]

If you look up the Javadoc specification of the Predicate interface, you may notice additional methods such as and and or.

### Consumer

The java.util.function.Consumer\<T> interface defines an abstract method named accept that takes an object of generic type T and returns no result (void).

You might use this interface when you need to access an object of type T and perform some oper- ations on it.

In [7]:
@FunctionalInterface
public interface Consumer<T> {
    void accept(T t);
}

public <T> void forEach(List<T> list, Consumer<T> c) {
	for (T t : list) {
		c.accept(t);
	}
}

In [8]:
forEach(Arrays.asList(1,2,3,4,5),
			(Integer i) -> System.out.println(i)
			);

1
2
3
4
5


### Function

The java.util.function.Function<T, R> interface defines an abstract method named apply that takes an object of generic type T as input and returns an object of generic type R.

You might use this interface when you need to define a lambda that maps information from an input object to an output (for example, extracting the weight of an apple or mapping a string to its length).

In [9]:
// @FunctionalInterface
// public interface Function<T, R> {
// 	R apply(T t);
// }
// Importing is same as writing the above, only difference is that other default methods will not be for free.
import java.util.function.Function;

public <T, R> List<R> map(List<T> list, Function<T, R> f) {
	List<R> result = new ArrayList<>();
	for (T t : list) {
		result.add(f.apply(t));
	}
	return result;
}

In [10]:
// [7, 2, 6]
List<Integer> l = map(Arrays.asList("lambdas", "in", "action"), (String s) -> s.length());

l

[7, 2, 6]

We described three functional interfaces that are generic: Predicate<T>, Consumer<T>, and Function<T, R>. There are also functional interfaces that are specialized with certain types.

<img src="img/Chapter_3_commonFunctions.png" style="height:500px">

__To summarize the discussion about functional interfaces and lambdas, table 3.3 provides a summary of use cases, examples of lambdas, and functional interfaces that can be used.__

<img src="img/Chapter_3_examples_of_lamdas.png" style="height:300px">

What about exceptions, lambdas, and functional interfaces?

You have two options :
* If you need the body of a lambda expression to throw an exception: define your own functional interface that declares the checked exception
* Wrap the lambda body with a try/catch block.

```
// First option

@FunctionalInterface
public interface BufferedReaderProcessor {
    String process(BufferedReader b) throws IOException;
}
BufferedReaderProcessor p = (BufferedReader br) -> br.readLine();

```

```
// Second Option

Function<BufferedReader, String> f = (BufferedReader b) -> {
try {
return b.readLine();
    }
    catch(IOException e) {
      throw new RuntimeException(e);
    }
};
```

The type of a lambda is deduced from the context in which the lambda is used. The type expected for the lambda expression inside the context (for example, a method parameter that it’s passed to or a local variable that it’s assigned to) is called the target type.

Because of the idea of target typing, the same lambda expression can be associated with different functional interfaces if they have a compatible abstract method signature.

The same lambda can be used with multi- ple different functional interfaces:

```
Comparator<Apple> c1 =(Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight());
ToIntBiFunction<Apple, Apple> c2 = (Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight());
BiFunction<Apple, Apple, Integer> c3 = (Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight());
```

They can get their target type from an assignment context, method-invocation context (parameters and return), and a cast context.

You’ve seen how the target type can be used to check whether a lambda can be used in a particular context. It can also be used to do something slightly different: infer the types of the parameters of a lambda.

__Type inference__

The Java compiler deduces what func- tional interface to associate with a lambda expression from its surrounding context (the target type), meaning it can also deduce an appropriate signature for the lambda because the function descriptor is available through the target type. 

In [11]:
Comparator<Apple> c1 = (Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight());
Comparator<Apple> c2 = (a1, a2) -> a1.getWeight().compareTo(a2.getWeight());

In [12]:
var inventory = getInventory();
inventory

[Apple{color='green', weight=80}, Apple{color='green', weight=255}, Apple{color='red', weight=220}]

In [13]:
inventory.sort(c2);
inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

Note that sometimes it’s more readable to include the types explicitly, and sometimes it’s more readable to exclude them. There’s no rule for which way is better; developers must make their own choices about what makes their code more readable.

__Using local variables__

But lambda expressions are also allowed to use free variables (variables that aren’t the parameters and are defined in an outer scope) like anonymous classes can. They’re called capturing lambdas. For example, the following lambda captures the vari- able portNumber:

```
int portNumber = 1337;
Runnable r = () -> System.out.println(portNumber);
```

Lambdas are allowed to capture (to reference in their bodies) instance variables and static variables without restrictions. But when local variables are captured, they have to be explicitly declared final or be effectively final.

In [14]:
int portNumber = 1337;
Runnable r = () -> System.out.println(portNumber); portNumber = 31337;

In [15]:
Thread t = new Thread(r);
t.start();

31337

## Method references

Method references let you reuse existing method definitions and pass them like lambdas. 

In [16]:
inventory = getInventory();
inventory

[Apple{color='green', weight=80}, Apple{color='green', weight=255}, Apple{color='red', weight=220}]

In [17]:
import static java.util.Comparator.comparing;
inventory.sort(comparing(Apple::getWeight));

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

__In a nutshell__

Indeed, a method reference lets you cre- ate a lambda expression from an existing method implementation. But by referring to a method name explicitly, your code can gain better readability.

You can think of method references as syntactic sugar for lambdas that refer only to a single method because you write less to express the same thing.

There are three main kinds of method references:
* A method reference to a static method (for example, the method parseInt of Integer, written Integer::parseInt)
* A method reference to an instance method of an arbitrary type (for example, the method length of a String, written String::length)
* A method reference to an instance method of an existing object or expression (for example, suppose you have a local variable expensiveTransaction that holds an object of type Transaction, which supports an instance method getValue; you can write expensiveTransaction::getValue)

In [18]:
private boolean isValidName(String string) {
    return Character.isUpperCase(string.charAt(0));
}

You can now pass this method around in the context of a Predicate<String> using a method reference:

```
filter(words, this::isValidName)
```


In [19]:
List<String> str = Arrays.asList("a","b","A","B");
str.sort((s1, s2) -> s1.compareToIgnoreCase(s2));

str

[a, A, b, B]

In [20]:
List<String> str = Arrays.asList("a","b","A","B");
str.sort(String::compareToIgnoreCase);

str

[a, A, b, B]

Note that the compiler goes through a similar type-checking process as for lambda expressions to figure out whether a method reference is valid with a given functional interface.

__Constructor references__

You can create a reference to an existing constructor using its name and the keyword new as follows: ClassName::new.

In [21]:
import java.util.function.Supplier;

Supplier<Apple> c1 = Apple::new;
Apple a1 = c1.get();

a1

Apple{color='white', weight=0}

In [22]:
import java.util.function.Function;
Function<Integer, Apple> c2 = Apple::new;
Apple a2 = c2.apply(110);

a2

Apple{color='', weight=110}

In [23]:
List<Integer> weights = Arrays.asList(7, 3, 4, 10);
List<Apple> apples = map(weights, Apple::new);

public List<Apple> map(List<Integer> list, Function<Integer, Apple> f) {
    List<Apple> result = new ArrayList<>();
    for(Integer i: list) {
        result.add(f.apply(i)); 
    }
    return result;
}

apples

[Apple{color='', weight=7}, Apple{color='', weight=3}, Apple{color='', weight=4}, Apple{color='', weight=10}]

In [24]:
import java.util.function.BiFunction;

BiFunction<Integer, String, Apple> c3 = Apple::new;
Apple a3 = c3.apply(110, "green");

a3

Apple{color='green', weight=110}

Because there isn’t one in the functional interface starter set, you can create your own for tri function.

In [25]:
public interface TriFunction<T, U, V, R> {
    R apply(T t, U u, V v);
}

The above function can be used in the case of RGB.

```
TriFunction<Integer, Integer, Integer, RGB> colorFactory = RGB::new;
```

__Putting lambdas and method references into practice__

In [26]:
inventory = getInventory();

__1st Attempt__

In [27]:
public class AppleComparator implements Comparator<Apple> {
                public int compare(Apple a1, Apple a2){
                    return a1.getWeight().compareTo(a2.getWeight());
                }
}

inventory.sort(new AppleComparator());

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

__2nd Attempt__

In [28]:
inventory = getInventory();

inventory.sort((Apple a1, Apple a2) -> a1.getWeight().compareTo(a2.getWeight()));
inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

__3rd Attempt__

In [29]:
inventory = getInventory();
inventory.sort((a1, a2) -> a1.getWeight().compareTo(a2.getWeight()));

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

__4th Attempt__

In [30]:
// Comparator<Apple> c = Comparator.comparing((Apple a) -> a.getWeight());

inventory = getInventory();

import static java.util.Comparator.comparing;
inventory.sort(comparing(apple -> apple.getWeight()));

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

__5th Attempt (Final: See the beauty of code.)__

In [31]:
inventory = getInventory();

inventory.sort(comparing(Apple::getWeight));

inventory

[Apple{color='green', weight=80}, Apple{color='red', weight=220}, Apple{color='green', weight=255}]

Congratulations, this is your final solution! Why is this better than code prior to Java 8? It’s not only because it’s shorter; it’s also obvious what it means. The code reads like the problem statement “sort inventory comparing the weight of the apples.”

__Composing Comparators__

The interface includes a default method reversed that reverses the ordering of a given comparator.

In [32]:
// Comparator<Apple> c = Comparator.comparing(Apple::getWeight);
// REVERSED ORDER

inventory = getInventory();
inventory.sort(comparing(Apple::getWeight).reversed());

inventory

[Apple{color='green', weight=255}, Apple{color='red', weight=220}, Apple{color='green', weight=80}]

CHAINING COMPARATORS

```
inventory.sort(comparing(Apple::getWeight) .reversed()
         .thenComparing(Apple::getCountry));
         ```

__Composing Predicates__

The Predicate interface includes three methods that let you reuse an existing Predicate to create more complicated ones: negate, and, and or.

In [33]:
// Predicate<Apple> notRedApple = redApple.negate();

__Composing Functions__

Finally, you can also compose lambda expressions represented by the Function inter- face. The Function interface comes with two default methods for this, andThen and compose, which both return an instance of Function.

For example, given a function f that increments a number (x -> x + 1) and another function g that multiples a number by 2, you can combine them to create a function h that first increments a number and then multiplies the result by 2:

In [34]:
Function<Integer, Integer> f = x -> x + 1; 
Function<Integer, Integer> g = x -> x * 2; 
Function<Integer, Integer> h = f.andThen(g); 
int result = h.apply(1);

result

4

You can also use the method compose similarly to first apply the function given as argu- ment to compose and then apply the function to the result. For example, in the previous example using compose, it would mean f(g(x)) instead of g(f(x)) using andThen:

In [35]:
Function<Integer, Integer> f = x -> x + 1; 
Function<Integer, Integer> g = x -> x * 2; 
Function<Integer, Integer> h = f.compose(g); 
int result = h.apply(1);

result

3

In [41]:
public class Letter{
    public static String addHeader(String text) {
        return "From Honey, Raoul, Mario and Alan: " + text;
    }
    public static String addFooter(String text) {
        return text + " Kind regards";
    }
    public static String checkSpelling(String text) {
        return text.replaceAll("labda", "lambda"); 
    }
}

In [42]:
Function<String, String> addHeader = Letter::addHeader;
Function<String, String> transformationPipelineWithSpellingCheck = addHeader.andThen(Letter::checkSpelling).andThen(Letter::addFooter);
String result = transformationPipelineWithSpellingCheck.apply("Testing labda!");

result

From Honey, Raoul, Mario and Alan: Testing lambda! Kind regards

In [44]:
Function<String, String> addHeader = Letter::addHeader;
Function<String, String> transformationPipeline = addHeader.andThen(Letter::addFooter);

String result = transformationPipeline.apply("Testing labda!");

result

From Honey, Raoul, Mario and Alan: Testing labda! Kind regards

## Summary

* A lambda expression can be understood as a kind of anonymous function: it doesn’t have a name, but it has a list of parameters, a body, a return type, and also possi- bly a list of exceptions that can be thrown.

* Lambda expressions let you pass code concisely.

* A functional interface is an interface that declares exactly one abstract method.

* Lambda expressions can be used only where a functional interface is expected.

* Lambda expressions let you provide the implementation of the abstract method of a functional interface directly inline and treat the whole expression as an instance of a functional interface.

* Java 8 comes with a list of common functional interfaces in the java.util.function package, which includes Predicate\<T>, Function\<T, R>, Supplier\<T>, Consumer\<T>, and BinaryOperator\<T>, described in table 3.2.

* Primitive specializations of common generic functional interfaces such as Predicate\<T> and Function\<T, R> can be used to avoid boxing operations: IntPredicate, IntToLongFunction, and so on.

* The execute-around pattern (for when you need to execute some given behav- ior in the middle of boilerplate code that’s necessary in a method, for example, resource allocation and cleanup) can be used with lambdas to gain additional flexibility and reusability.

* The type expected for a lambda expression is called the target type.

* Method references let you reuse an existing method implementation and pass it around directly.

* Functional interfaces such as Comparator, Predicate, and Function have several default methods that can be used to combine lambda expressions.